In [1]:
#读取数据
import torch
import os
import torchvision
def load_data():
    dataset=torchvision.datasets.FashionMNIST(root="fashion-mnist",train=False,transform=torchvision.transforms.ToTensor(),download=True)
    return dataset
#初始化数据集
dataset=load_data()
len(dataset),dataset,type(dataset)

C:\Users\ROG\.conda\envs\pytorch2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(10000,
 Dataset FashionMNIST
     Number of datapoints: 10000
     Root location: fashion-mnist
     Split: Test
     StandardTransform
 Transform: ToTensor(),
 torchvision.datasets.mnist.FashionMNIST)

In [2]:
#数据集加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=32,
                                     shuffle=True,
                                     drop_last=True)

x, y = next(iter(loader))
len(loader), x.reshape(-1,784).shape, y

(312,
 torch.Size([32, 784]),
 tensor([6, 6, 0, 1, 0, 1, 7, 2, 8, 3, 1, 3, 1, 0, 8, 2, 9, 7, 5, 2, 8, 9, 2, 4,
         5, 2, 2, 5, 5, 5, 4, 8]))

In [3]:
#nn.Linear实现RNN神经网络
class RNN(torch.nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.u = torch.nn.Linear(784, 256)#定义输入层
        self.w = torch.nn.Linear(256, 256)#定义隐藏层
        self.v = torch.nn.Linear(256, 10)#定义输出层
        self.tanh = torch.nn.Tanh()#激活函数
        self.LogSoftmax = torch.nn.LogSoftmax(dim=1)#定义归一化函数，对应了torch.nn.NLLLoss()

    def forward(self, inputs, hidden):
        
        u_x = self.u(inputs)#输入层
        hidden = self.w(hidden)#获取上一个隐藏层
        hidden = self.tanh(hidden + u_x)#隐藏层和输入相加传入激活函数，并更新隐藏层
        output = self.LogSoftmax(self.v(hidden))#归一化输出
        
        return output, hidden

    def initHidden(self):#初始化隐藏层
        return torch.zeros(1, 256)

rnn=RNN()
rnn

RNN(
  (u): Linear(in_features=784, out_features=256, bias=True)
  (w): Linear(in_features=256, out_features=256, bias=True)
  (v): Linear(in_features=256, out_features=10, bias=True)
  (tanh): Tanh()
  (LogSoftmax): LogSoftmax(dim=1)
)

In [4]:
#训练
rnn.cuda()
import tqdm
def train():
    optimizer = torch.optim.Adam(rnn.parameters(), lr=1e-4)
    
    criterion = torch.nn.NLLLoss()#
    rnn.train()
    
    bar=tqdm.tqdm(range(5))
    hidden=rnn.initHidden().cuda()
    hidden = hidden.cuda()
    
    for epoch in bar:
        #rnn.zero_grad()
        for i ,( x , y )in enumerate(loader):
            x=x.reshape(-1,784).cuda()
            y=y.cuda()
            output, hidden = rnn(x, hidden)
            loss = criterion(output,y)
            loss.backward(retain_graph=True)
            """在PyTorch中，当你执行.backward()方法时，计算图上的所有中间张量都会被释放，
            以节省内存。如果你需要多次进行反向传播，或者在反向传播后需要访问这些中间张量，
            你需要设置retain_graph=True参数。"""
            for p in rnn.parameters():
                #p.data.add_(-learning_rate, p.grad.data)
                p.data-=learning_rate*p.grad.data
            optimizer.zero_grad()
        #求一个batch内各指标并累加
        target_label=7#自定义正例，其他为负例
        TP=TN=FP=FN=0
        j=0
        for x in output.argmax(dim=1):
            if x==target_label and y[j]==target_label:
                TP+=1
            elif x==target_label and y[j]!=target_label:
                FP+=1
            elif x!=target_label and y[j]!=target_label:
                TN+=1
            else:
                FN+=1
            j+=1
        accuracy=(TP+TN)/(TP+TN+FP+FN)if (TP+TN+FP+FN) > 0 else 0
        precision=TP/(TP+FP)if (TP + FP) > 0 else 0
        recall=TP/(TP+FN)if (TP + FN) > 0 else 0
        F1=2*(precision*recall)/(precision+recall)if (precision+recall) > 0 else 0
        print(epoch, loss.item())
        print(f"accuracy={accuracy}")
        print(f"precision={precision}")
        print(f"recall={recall}")
        print(f"F1={F1}")

            # Add parameters' gradients to their values, multiplied by learning rate
    return output, loss.item()

In [5]:
learning_rate = 0.001
train()

 20%|███████████▊                                               | 1/5 [00:10<00:43, 10.92s/it]

0 2.049337387084961
accuracy=0.9375
precision=0.5
recall=1.0
F1=0.6666666666666666


 40%|███████████████████████▌                                   | 2/5 [00:39<01:03, 21.29s/it]

1 1.8257420063018799
accuracy=0.90625
precision=0.4
recall=1.0
F1=0.5714285714285715


 60%|███████████████████████████████████▍                       | 3/5 [01:25<01:05, 32.68s/it]

...

 80%|███████████████████████████████████████████████▏           | 4/5 [02:29<00:44, 44.93s/it]

3 1.4492310285568237
accuracy=0.96875
precision=0.8
recall=1.0
F1=0.888888888888889


100%|███████████████████████████████████████████████████████████| 5/5 [03:54<00:00, 46.86s/it]

4 1.4595184326171875
accuracy=0.9375
precision=0.5
recall=1.0
F1=0.6666666666666666


(tensor([[-2.7020, -3.7757, -1.3965, -3.3789, -1.2160, -4.3824, -1.8359, -4.3601,
          -2.1561, -3.4568],
         [-3.2237, -4.0865, -2.2417, -3.3017, -1.7495, -2.9610, -2.5646, -2.6824,
          -1.3594, -1.7574],
         [-2.3162, -3.9882, -1.3741, -3.4135, -1.5232, -4.5005, -1.6902, -4.6081,
          -2.0098, -3.2382],
         [-1.0516, -2.2144, -2.2714, -2.0305, -2.6347, -3.9476, -2.0229, -4.2385,
          -3.1694, -3.6041],
         [-2.1569, -2.5852, -1.9317, -2.1133, -1.4627, -4.1311, -1.7927, -4.1966,
          -2.5725, -3.2802],
         [-3.9851, -4.5228, -3.1957, -3.8994, -3.3334, -2.5952, -3.5379, -2.9918,
          -2.2922, -0.4801],
         [-2.5842, -3.2309, -1.4868, -2.9565, -1.5768, -3.3446, -1.9271, -3.3343,
          -1.9693, -3.1179],
         [-2.8325, -2.8855, -2.7444, -2.5483, -2.7624, -1.8279, -2.6773, -1.7305,
          -2.1155, -1.8814],
         [-1.5340, -1.9725, -2.3816, -1.5976, -2.0983, -4.1551, -2.0463, -4.2279,
          -3.1794, -3.6238],
 

In [6]:
torch.save(rnn, 'rnn_model')

In [11]:
#测试
@torch.no_grad()
def test():
    model = torch.load('rnn_model')
    model.eval()
    model=model.cuda()
    hidden=rnn.initHidden()

    target_label=7#自定义正例，其他为负例
    TP=TN=FP=FN=0
    #遍历整个数据集
    for i, (x, y) in enumerate(loader):
        x=x.reshape(-1,784).cuda()
        y=y.cuda()
        hidden=hidden.cuda()
        out,hidden = rnn(x,hidden)
        #求一个batch内各指标并累加
        j=0
        for x in out.argmax(dim=1):
            if x==target_label and y[j]==target_label:
                TP+=1
            elif x==target_label and y[j]!=target_label:
                FP+=1
            elif x!=target_label and y[j]!=target_label:
                TN+=1
            else:
                FN+=1
            j+=1
    accuracy=(TP+TN)/(TP+TN+FP+FN)if (TP+TN+FP+FN) > 0 else 0
    precision=TP/(TP+FP)if (TP + FP) > 0 else 0
    recall=TP/(TP+FN)if (TP+FN) > 0 else 0
    F1=2*(precision*recall)/(precision+recall)if (precision+recall) > 0 else 0
    print(f"accuracy={accuracy}")
    print(f"precision={precision}")
    print(f"recall={recall}")
    print(f"F1={F1}")
test()

accuracy=0.9331931089743589
precision=0.6152234636871509
recall=0.8836509528585758
F1=0.7254013997529848
